In [ ]:
# este es el bueno
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def construccion_pd(ruta_archivo):
    """
    Construimos la funcion apartir de el archivo Excel para extraer la
    informacion y obtener los nodos y arcos de la grafica
    """
    df = pd.read_excel(ruta_archivo) #Leer el Excel con ayuda de la paqueteria pandas
    nodos = list(range(len(df["Index"])))
    """Creamos la lista "nodos" con los datos de la columna
    "Index" del archivo excel """
    arcos = [] # Se inicializa la lista para almacenar los arcos
    for i, cited_by in enumerate(df["Cited by"]):
    # Construimos los arcos iterando sobre la columna "Cited by"
        for j in cited_by.split(','):# Separamos los  nodos
            arcos.append((int(j) - 1, i)) # Agremos los arcos como tuplas
    return Red(nodos, arcos)

class Grafica:
  # Contruimos la clase Grafuca con los nodos y arcos
    def __init__(self, nodos, arcos):
        self.matriz = np.zeros((len(nodos), len(nodos)))
        # Creamos una matriz inicializadada con puros ceros
        for x in arcos:
            self.matriz[x[0], x[1]] = 1 # Definimos los arcos en la matriz

    def __str__(self):
      # epresentamos la grafica como cadena
        return str(self.matriz)

    def agregar_arcos(self, tupla):
      # Metodo para agregar arcos
        self.matriz[tupla[0], tupla[1]] = 1

    def quitar_arcos(self, tupla):
      # Metodo para quitar arcos
        self.matriz[tupla[0], tupla[1]] = 0

    def num_nodos(self):
        return len(self.matriz)

    def num_arcos(self):
        return int(self.matriz.sum())

    def agregar_nodo(self):
      # Metodo para agregar un arco a la grafica
        vertical = np.zeros((len(self.matriz), 1))
        horizontal = np.zeros((1, len(self.matriz) + 1))
        self.matriz = np.vstack((np.hstack((self.matriz, vertical)), horizontal))

    def quitar_nodo(self, nodo):
      # Metodo para eliminar un arco de la grafica
        self.matriz = np.delete(self.matriz, nodo, axis=0)
        self.matriz = np.delete(self.matriz, nodo, axis=1)

    def revisar_conexidad(self):
      #Verifica que los nodos tengan una conexion saliente
      for i in range(len(self.matriz)):
        if self.matriz[i,:].sum() == 0:
        # Si un nodo no teiene conexion saliente se retorna True y
        #False en caso contario
          return False
      return True

    def revisar_ciclicidad(self): # Verifica la ciclicidad de la grafica
      for i in range(len(self.matriz)):
        if self.matriz[i,i] == 1:
          # Si tiene ciclos retorna True y False de caso contario
          return True
      return False

    def num_adyacentes(self, nodo):
      # Numero de conexiones salientes a un nodo
      return self.matriz[nodo,:].sum()


class Red(Grafica):
  # Clase que hereda de grafica
    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)# Llama al contructor base

    def matriz_transicion(self):
    # Creamos la matriz de transicion
        transicion = np.zeros_like(self.matriz)
        for i in range(len(self.matriz)):
            transicion[i, :] = self.matriz[i, :] / self.matriz[i, :].sum()
        return transicion

    def calcular_pagerank(self, d=0.85, epsilon=1e-6):
      # Metodo para calcular el PageRank de los nodos
       n = self.num_nodos()
       pagerank = np.ones(n) / n # Vector de PageRank inicial
       M = self.matriz_transicion()
       U = np.ones((n, n)) / n  # Se define  la  Matriz uniforme
       M = d * M + (1 - d) * U # Redefine la matriz de transicion
       pagerank = pi_0

       while True:
        # Multiplicacion del vector de PageRank por la matriz de transicion
         nuevo_pagerank = pagerank @ M
        # Verificamos la convergencia
         if np.linalg.norm(nuevo_pagerank - pagerank, 1) < epsilon:
           break
        # Se actualiza el vector de PagaRank con el nuevo valor
         pagerank = nuevo_pagerank
       return pagerank

if __name__ == "__main__":
    # Ruta al archivo de datos
    ruta_archivo = "/content/drive/MyDrive/Web.xlsx"
    df = pd.read_excel(ruta_archivo)

    grafica = construccion_pd(ruta_archivo)
    print("Matriz de la gráfica:")
    print(grafica.matriz)

    matriz_transicion = grafica.matriz_transicion()
    print("Matriz de transiciones:")
    print(matriz_transicion)
#pregunta 1
    # Vector de  proba inicial uniforme
    pi = np.ones((1, len(matriz_transicion)))/len(matriz_transicion)
    # Se actualiza el vector de probabilidad
    pi_1 = pi@matriz_transicion

    while np.linalg.norm(pi - pi_1) > 1e-6:
      pi = pi_1
      # Se multiplica la matriz de transicion por el vector de proba
      pi_1 = pi@matriz_transicion
    print("Pregunta 1:")
    print(pi)

#pregunta 2
    print("Pregunta 2")
    if "Website" in df.columns:
      # Leemos la columa "Website" en el archivo para extraer la informacion de
      # las pagunas web con terminacon ".ru"
      nodos_ru = df[df["Website"].str.contains(".ru", na=False)].index.tolist()
      print(f"Nodos con dominio '.ru': {nodos_ru}")
    else:
      print("La columna 'Website' no se encuentra en el archivo.")
      nodos_ru = [] # Agregamos a una lista los nodos con terminacion ".ru"

    # Crear vector inicial personalizado
    n = grafica.num_nodos()
    pi_0 = np.ones(n)  * 0.3 / n  # Asignar valor 0.3 entre todos los nodos
    if nodos_ru:
     for nodo in nodos_ru:
       pi_0[nodo] += 0.7 / len(nodos_ru)  # Incrementar 0.7 del valor para nodos ".ru"
    print("\nVector inicial modificado para '.ru':")
    print(pi_0)

    # Calcular PageRank con vector inicial modificado
    pagerank_modificado = grafica.calcular_pagerank(d=0.85)
    print("PageRank con vector inicial modificado (.ru):")
    print(pagerank_modificado)

    # Calcular PageRank con vector inicial uniforme
    pagerank_uniforme = grafica.calcular_pagerank(d=0.85)
    print("\nPageRank con vector inicial uniforme:")
    print(pagerank_uniforme)
#pregunta 3
    print("Pregunta 3")
    for d in [0.5, 0.85, 1.0]:
        print(f"\nResultados de PageRank para d = {d}:")
        pagerank = grafica.calcular_pagerank(d=d)

        print(pagerank)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Matriz de la gráfica:
[[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 1. 0. 0. 0.